In [5]:
import re
import time
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup


In [ ]:
# Initialize the driver
driver = webdriver.Chrome()
driver.get("https://www.instagram.com/")

# Login
time.sleep(5)
username = driver.find_element(By.CSS_SELECTOR, "input[name='username']")
password = driver.find_element(By.CSS_SELECTOR, "input[name='password']")
username.clear()
password.clear()
username.send_keys("        ")
password.send_keys("        ")
login = driver.find_element(By.CSS_SELECTOR, "button[type='submit']").click()
time.sleep(5)

# Go directly to the profile page
print("📍 الانتقال إلى صفحة البروفايل..")
time.sleep(3)
driver.get("https://www.instagram.com/l3gq/")
print("⏳ انتظار تحميل الصفحة...")
time.sleep(3)  # انتظار 3 ثواني لتحميل الصفحة بالكامل
print("✅ تم تحميل الصفحة بنجاح!")
print("🔄 بدء جمع روابط البوستات...")

📍 الانتقال إلى صفحة البروفايل..
⏳ انتظار تحميل الصفحة...
⏳ انتظار تحميل الصفحة...
✅ تم تحميل الصفحة بنجاح!
🔄 بدء جمع روابط البوستات...
✅ تم تحميل الصفحة بنجاح!
🔄 بدء جمع روابط البوستات...


In [10]:

# --- جمع روابط البوستات مع سكرول ---
all_posts = set()
no_change = 0
last_height = driver.execute_script("return document.body.scrollHeight")

while no_change < 3:
    links = driver.find_elements(By.TAG_NAME, 'a')
    for link in links:
        href = link.get_attribute('href')
        if href and ('/p/' in href or '/reel/' in href):
            all_posts.add(href)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        no_change += 1
    else:
        no_change = 0
    last_height = new_height

posts = list(all_posts)
print(f"✅ تم جمع {len(posts)} رابط بوست.")

# --- حفظ الروابط في ملف CSV ---
df_posts = pd.DataFrame({
    'Post Number': range(1, len(posts) + 1),
    'Post Link': posts
})

posts_filename = f"collected_post_links_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df_posts.to_csv(posts_filename, index=False, encoding='utf-8-sig')
print(f"💾 تم حفظ روابط البوستات في: {posts_filename}")
print(f"✅ الملف يحتوي على {len(posts)} رابط")


✅ تم جمع 36 رابط بوست.
💾 تم حفظ روابط البوستات في: collected_post_links_20251115_005515.csv
✅ الملف يحتوي على 36 رابط


In [12]:
# Helper: parse compact like strings ("1.2K", "3M", "1,234") and robustly extract likes from the post DOM
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException

def _parse_compact_number(s):
    """Convert strings like '1,234', '1.2K', '3M' to int. Returns None if can't parse."""
    if not s:
        return None
    s = s.strip()
    # try direct integer with commas
    try:
        return int(s.replace(",", ""))
    except Exception:
        pass

    # compact form like 1.2K, 3M, 800k
    m = re.match(r"^([\d\.]+)\s*([kKmM])$", s)
    if m:
        val = float(m.group(1))
        suf = m.group(2).lower()
        if suf == 'k':
            return int(val * 1_000)
        if suf == 'm':
            return int(val * 1_000_000)
    # fallback: extract first plain number with commas or dots
    m2 = re.search(r"([\d,]+)(?:\.|,\d+)?", s)
    if m2:
        try:
            return int(m2.group(1).replace(',', ''))
        except Exception:
            return None
    return None


def get_likes_count(driver, root=None, timeout=2, verbose=False):
    """
    Robustly extract the likes count from a post page.

    Args:
        driver: Selenium WebDriver instance.
        root: optional WebElement to scope the search (e.g., article element). If None, search the whole page.
        timeout: not used for explicit wait here but kept for compatibility.
        verbose: if True prints debug messages.

    Returns:
        int likes_count or None if not found/parsable.
    """
    search_root = root if root is not None else driver

    strategies = []

    # 1) anchor to liked_by (most reliable in the snippet you provided)
    strategies.append((By.XPATH, ".//a[contains(@href, '/liked_by')]") if root is not None else (By.XPATH, "//a[contains(@href, '/liked_by')]") )

    # 2) common spans/buttons inside section that show like counts
    strategies.append((By.XPATH, ".//section//span[contains(., 'likes') or contains(., 'like')]") if root is not None else (By.XPATH, "//section//span[contains(., 'likes') or contains(., 'like')]") )

    # 3) any span/button with numeric text near role='button'
    strategies.append((By.XPATH, ".//span[@role='button' and string-length(normalize-space(.))>0]") if root is not None else (By.XPATH, "//span[@role='button' and string-length(normalize-space(.))>0]") )

    # 4) fallback: any element that contains word 'likes' and a number
    strategies.append((By.XPATH, ".//*[contains(translate(., 'LIKES', 'likes'), 'likes')]") if root is not None else (By.XPATH, "//*[contains(translate(., 'LIKES', 'likes'), 'likes')]") )

    for by, expr in strategies:
        try:
            if verbose:
                print('Trying strategy:', by, expr)
            elems = search_root.find_elements(by, expr)
            for el in elems:
                # prefer visible text
                text = ''
                try:
                    text = el.text.strip()
                except Exception:
                    try:
                        text = el.get_attribute('innerText') or el.get_attribute('textContent') or ''
                    except Exception:
                        text = ''

                if not text:
                    # try attributes that sometimes contain counts
                    for attr in ('aria-label', 'title', 'data-count'):
                        try:
                            val = el.get_attribute(attr)
                            if val:
                                text = val
                                break
                        except Exception:
                            continue

                if not text:
                    # try nested span
                    try:
                        sp = el.find_element(By.XPATH, ".//span")
                        text = sp.text.strip() if sp.text else (sp.get_attribute('innerText') or '')
                    except Exception:
                        pass

                if not text:
                    continue

                # extract digits or compact formats
                # first try to find a number next to 'like' word
                # examples: '32 likes', '1,234 likes', '1.2K likes'
                m = re.search(r"([\d\.,]+\s*[kKmM]?)", text)
                if m:
                    candidate = m.group(1).strip()
                    parsed = _parse_compact_number(candidate)
                    if parsed is not None:
                        if verbose:
                            print('Parsed', candidate, '->', parsed)
                        return parsed

        except Exception as e:
            if verbose:
                print('strategy error:', e)
            continue

    # final fallback: search whole page text for pattern like '123 likes'
    try:
        page_text = driver.page_source
        m = re.search(r"([\d\.,]+\s*[kKmM]?)\s*(?:likes|like|إعجابات|إعجاب)", page_text)
        if m:
            parsed = _parse_compact_number(m.group(1))
            if parsed is not None:
                return parsed
    except Exception:
        pass

    if verbose:
        print('Likes count not found')
    return None


In [13]:
# --- الدالة الجديدة لاستخراج الهشتاجات ---
def extract_hashtags(caption_text):
    """
    دالة لاستخراج الهشتاجات من نص الكابشن باستخدام regex.

    Args:
        caption_text: نص الكابشن.

    Returns:
        سلسلة نصية مفصولة بمسافات تحتوي على الهشتاجات فقط.
    """
    return ' '.join(re.findall(r'#\w+', caption_text)) if caption_text else ''


# --- الدالة الجديدة لاستخراج منشن ---
def extract_captions(caption_text):
    """
    دالة لاستخراج الهشتاجات من نص الكابشن باستخدام regex.

    Args:
        caption_text: نص الكابشن.

    Returns:
        سلسلة نصية مفصولة بمسافات تحتوي على الهشتاجات فقط.
    """
    return ' '.join(re.findall(r'@\w+', caption_text)) if caption_text else ''


In [ ]:
if posts:
    # إنشاء هيكل البيانات الرئيسي
    all_posts_data = {
        "Post Number": [],
        "Post Link": [],
        "Media Type": [],
        "Post Date": [],
        "Likes Count": [],
        "Caption": [],
        "Hashtags": [],
        "Comments": [],
        "Comment Count": []
    }

    # Loop على كل البوستات
    for post_index, post_url in enumerate(posts, start=1):
        print(f"\n{'=' * 80}")
        print(f"🔗 فتح البوست رقم {post_index} من {len(posts)}: {post_url}")
        print(f"{'=' * 80}\n")

        driver.get(post_url)
        time.sleep(3)
        wait = WebDriverWait(driver, 10)

        try:
            wait.until(EC.presence_of_element_located((By.XPATH, "//article | //div[contains(@class, 'x1n2onr6')]")))
            print("✅ تم تحميل صفحة البوست.")
        except TimeoutException:
            print("⚠️ ربما تأخر تحميل البوست، استمرار المحاولة...")

        # ============================================
        # 📊 جمع البيانات من البوست المفتوح
        # ============================================

        # إضافة رقم البوست والرابط
        all_posts_data["Post Number"].append(post_index)
        all_posts_data["Post Link"].append(post_url)

        # ======================================================================================================

        # --- 1. Post Date ---
        try:
            time_element = wait.until(EC.presence_of_element_located((By.XPATH, "//time[@datetime]")))
            post_date = time_element.get_attribute('datetime')
            all_posts_data["Post Date"].append(post_date)
            print(f"📅 Post Date: {post_date}")
        except TimeoutException:
            print("❌ Post Date: Not Found")
            all_posts_data["Post Date"].append("N/A")

        # ======================================================================================================

        # --- 2. Media Type ---
        try:
            video_element = driver.find_elements(By.TAG_NAME, "video")
            if video_element:
                media_type = "Video"
            else:
                img_element = driver.find_elements(By.TAG_NAME, "img")
                if img_element:
                    media_type = "Image"
                else:
                    media_type = "Unknown"
            all_posts_data["Media Type"].append(media_type)
            print(f"🖼️ Media Type: {media_type}")
        except NoSuchElementException:
            print("❌ Media Type: Not Found")
            all_posts_data["Media Type"].append("N/A")

        # ======================================================================================================

        # --- 3. Caption & Hashtags (محدث لالتقاط الكابشن الكامل) ---
        caption = ''
        captions = ''
        hashtags = ''

        try:
            # 🎯 المحاولة الأولى: استهداف مباشر باستخدام XPath دقيق
            # نبحث عن span يحتوي على نص عربي أو هاشتاق داخل هيكل شائع في منشورات إنستغرام
            caption_element = driver.find_element(
                By.XPATH,
                "//div[contains(@class, 'x9f619') and contains(@class, 'xjbqb8w')]//span[contains(text(), '#') or contains(text(), '@')]"
            )
            caption = caption_element.text.strip()
            print(f"✅ الكابشن (بالاستهداف المباشر): {caption[:200]}...")

        except Exception as e1:
            print(f"⚠️ فشل الاستهداف المباشر: {e1}")

            try:
                # 🔄 المحاولة الثانية: البحث العام عن span طويل أو يحتوي على هاشتاق
                all_spans = driver.find_elements(By.TAG_NAME, "span")
                full_caption = ''

                # أولًا: ابحث عن span يحتوي على هاشتاق (الأكثر دقة)
                for span in all_spans:
                    text = span.text.strip()
                    if text and '#' in text:
                        full_caption = text
                        break

                # ثانيًا: إذا لم يُعثر، ابحث عن span طويل (> 30 حرفًا)
                if not full_caption:
                    for span in all_spans:
                        text = span.text.strip()
                        if text and len(text) > 30:
                            full_caption = text
                            break

                # ثالثًا: إذا فشل كل شيء، خذ آخر span غير فارغ معقول الطول
                if not full_caption:
                    for span in reversed(all_spans):
                        text = span.text.strip()
                        if text and len(text) > 10:
                            full_caption = text
                            break

                caption = full_caption
                print(f"✅ الكابشن (بالبحث العام): {caption[:200]}..." if caption else "❌ لم يتم العثور على كابشن.")

            except Exception as e2:
                print(f"❌ فشل جميع محاولات جلب الكابشن: {e2}")
                caption = ''

        # --- استخراج الهشتاجات والمنشنات باستخدام الدالة ---
        hashtags = extract_hashtags(caption)
        captions = extract_captions(caption)
        print(f"✅ الهشتاجات: {hashtags if hashtags else 'N/A'}")
        print(f"✅ المنشنات: {captions if captions else 'N/A'}")

        # --- حفظ في هيكل البيانات ---
        all_posts_data["Caption"].append(captions if captions else "No captions")
        all_posts_data["Hashtags"].append(hashtags if hashtags else "No hashtags")

        # ======================================================================================================

        # --- 4. Likes Count (محدث) ---
        try:
            # use the robust helper we added earlier
            likes_count = get_likes_count(driver, root=None, timeout=2, verbose=False)
            likes_count = str(likes_count) if likes_count is not None else "N/A"
            all_posts_data["Likes Count"].append(likes_count)
            print(f"❤️ Likes Count: {likes_count}")

        except Exception as e:
            print(f"❌ Likes Count Error: {str(e)}")
            all_posts_data["Likes Count"].append("N/A")

        # ======================================================================================================

        # --- 5. جمع التعليقات (Comments) مع السكرول لتحميل الكل ---
        comments_set = set()
        num_comments = 0

        try:
            print("🔄 بدء جمع التعليقات مع السكرول...")

            # محاولة الضغط على "عرض جميع التعليقات"
            try:
                view_all = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((
                        By.XPATH,
                        "//button[contains(text(), 'View all') or contains(text(), 'View all 1 replies') or contains(text(), 'View all 2 replies')]"
                    ))
                )
                view_all.click()
                time.sleep(2)
                print("✅ تم النقر على 'عرض المزيد'")
            except:
                print("ℹ️ لا يوجد زر 'عرض المزيد'")
                pass

            # العثور على حاوية التعليقات (إن وُجدت)
            comments_container = None

            # محاولة العثور على حاوية التعليقات الفعلية باستخدام جزء من الـ class المميز
            try:
                comments_container = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((
                        By.XPATH,
                        "//div[contains(@class, 'x5yr21d') and contains(@class, 'xw2csxc') and contains(@class, 'x1odjw0f')]"
                    ))
                )
                print("✅ تم العثور على حاوية التعليقات الجديدة")
            except TimeoutException:
                print("⚠️ لم يتم العثور على حاوية التعليقات")
                comments_container = None

            # --- السكرول وجمع الكومنتات بشكل تدريجي ---
            last_height = 0
            same_height = 0
            max_same = 5  # عدد المحاولات المتتالية قبل التوقف
            scroll_count = 0

            while same_height < max_same:
                scroll_count += 1
                print(f"   📜 سكرول رقم {scroll_count}...")

                # تنفيذ السكرول داخل الحاوية
                if comments_container:
                    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", comments_container)
                else:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

                # انتظار التحميل
                time.sleep(4)

                # جمع الكومنتات الحقيقية فقط (اللي فيها style و line-height = 20px)
                comment_spans = driver.find_elements(
                    By.XPATH,
                    "//span[@dir='auto' and @style and contains(@style, '20px')]"
                )

                # حفظ عدد التعليقات قبل الجمع الجديد
                old_count = len(comments_set)

                for span in comment_spans:
                    text = span.text.strip()
                    if text and len(text) > 0:
                        # تصفية إضافية: تجنب النصوص اللي فيها "like" أو "Reply"
                        if not any(kw in text for kw in
                                   ["like", "Reply", "See translation", "View all", "Messages", "Home", "Search",
                                    "Create", "Notifications", "Profile", "Also from Meta", "More", "Explore",
                                    "Reels"]):
                            comments_set.add(text)

                # حساب عدد التعليقات الجديدة المضافة
                new_count = len(comments_set)
                new_comments_added = new_count - old_count

                print(f"   ✅ تم جمع {new_comments_added} تعليق جديد | الإجمالي: {new_count}")

                # تحقق من الارتفاع لتحديد النهاية
                if comments_container:
                    new_height = driver.execute_script("return arguments[0].scrollHeight;", comments_container)
                else:
                    new_height = driver.execute_script("return document.body.scrollHeight;")

                # إذا الارتفاع لم يتغير، زيادة العداد
                if new_height == last_height:
                    same_height += 1
                    print(f"   ⚠️ الارتفاع لم يتغير ({same_height}/{max_same})")
                else:
                    same_height = 0  # إعادة تعيين العداد

                last_height = new_height

            comments_list = list(comments_set)
            num_comments = len(comments_list)
            comments_joined = ' | '.join(comments_list) if comments_list else "No comments"
            print(f"✅ تم جمع {num_comments} تعليق إجمالي بعد {scroll_count} عملية سكرول")

        except Exception as e:
            print(f"⚠️ خطأ في جمع التعليقات: {e}")
            comments_joined = "No comments"
            num_comments = 0

        # --- إضافة التعليقات إلى all_posts_data ---
        all_posts_data["Comments"].append(comments_joined)
        all_posts_data["Comment Count"].append(num_comments)

        print(f"\n✅ تم الانتهاء من البوست رقم {post_index}")

    # ======================================================================================================

    # بعد الانتهاء من كل البوستات
    print(f"\n{'=' * 80}")
    print(f"🎉 تم الانتهاء من جمع بيانات جميع البوستات ({len(posts)} بوست)")
    print(f"{'=' * 80}\n")

    # --- إنشاء DataFrame وطباعته ---
    df_all_posts = pd.DataFrame(all_posts_data)
    print("\n📊 تم جمع البيانات من جميع البوستات:")
    print(df_all_posts.to_string())

    # --- حفظ بيانات جميع البوستات ---
    all_posts_filename = f"all_posts_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_all_posts.to_csv(all_posts_filename, index=False, encoding='utf-8-sig')
    print(f"\n💾 تم حفظ بيانات جميع البوستات في: {all_posts_filename}")
    print(f"✅ الملف يحتوي على {len(df_all_posts)} صف و {len(df_all_posts.columns)} عمود")
    print(f"✅ إجمالي التعليقات المجموعة: {df_all_posts['Comment Count'].sum()}")
else:
    print("❌ لا توجد منشورات لمعالجتها.")
